In [77]:
import locale
import pandas as pd
import numpy as np
from scipy import stats

def formatear_moneda(valor):
    """
    Formatea un valor numérico como una cadena de texto con formato monetario.

    :param valor: El valor numérico a formatear.
    :param locale_str: La configuración regional a usar para el formateo.
    :return: Una cadena de texto con el valor formateado como moneda.
    """
    locale.setlocale(locale.LC_ALL, '')
    return locale.currency(valor, grouping=True)


def compute_stats(df, column, alpha = 0.95, formatter=lambda x:x):
    column = df[column]

    mean = column.mean()
    min = column.min()
    max = column.max()
    std = column.std()

    n = len(column)
    stderr = std / np.sqrt(n)
    h = stderr * stats.t.ppf((1 + alpha) / 2, n - 1)
    bottom_top_interval = mean - h
    top_interval = mean + h

    return {
        "desviación estándar": formatter(std),
        "mínimo": formatter(min),
        "mínimo del intervalo de confianza": formatter(bottom_top_interval),
        "media": formatter(mean),
        "máximo del intervalo de confianza": formatter(top_interval),
        "máximo": formatter(max)       
    }


In [59]:
df = pd.read_csv('clients.csv')
df.head()

,Unnamed: 0,simulation,type,arrive_time,requirement,waiting_room_arrive_time,ticker,price,leave_time,leakage_time,worker_helper
0,0,0,B,9.014119,1.0,9.020386,1.0,257646.313739,9.036567,NaN,0.0
1,1,0,A,9.017446,3.0,9.025951,1.0,0.000000,9.095540,NaN,1.0
2,2,0,A,9.040431,3.0,9.045977,2.0,0.000000,9.103548,NaN,1.0
3,3,0,A,9.094376,3.0,9.100651,3.0,0.000000,9.208577,NaN,1.0
4,4,0,B,9.122431,1.0,9.127099,2.0,148443.346508,9.277718,NaN,0.0


In [60]:
df['leakage'] = df['leave_time'].isna()

df['system_time'] = np.where(
    ~df['leakage'], 
    df['leave_time'] - df['arrive_time'], 
    df['leakage_time'] - df['arrive_time']
)

df['waiting_time'] = np.where(
    ~df['waiting_room_arrive_time'].isna(), 
    df['system_time'] + df['arrive_time'] - df["waiting_room_arrive_time"], 
    df['waiting_room_arrive_time']
)

df.head()

,Unnamed: 0,simulation,type,arrive_time,requirement,waiting_room_arrive_time,ticker,price,leave_time,leakage_time,worker_helper,leakage,system_time,waiting_time
0,0,0,B,9.014119,1.0,9.020386,1.0,257646.313739,9.036567,NaN,0.0,False,0.022448,0.016181
1,1,0,A,9.017446,3.0,9.025951,1.0,0.000000,9.095540,NaN,1.0,False,0.078093,0.069588
2,2,0,A,9.040431,3.0,9.045977,2.0,0.000000,9.103548,NaN,1.0,False,0.063117,0.057572
3,3,0,A,9.094376,3.0,9.100651,3.0,0.000000,9.208577,NaN,1.0,False,0.114201,0.107926
4,4,0,B,9.122431,1.0,9.127099,2.0,148443.346508,9.277718,NaN,0.0,False,0.155287,0.150619


In [67]:
group = df.groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total = {"total de clientes fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados,3.734115,9.662921,22.724064,22.797261,22.870457,37.887067


In [69]:
group = df[df['type'] == "A"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_A = {"total de clientes de tipo A fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo A fugados,5.488518,7.623318,25.631532,25.739118,25.846704,48.235294


In [71]:
group = df[df['type'] == "B"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_B = {"total de clientes de tipo B fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo B fugados,4.012501,0.0,11.89675,11.975404,12.054057,28.682171


In [72]:
group = df[df['type'] == "C"].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_C = {"total de clientes de tipo C fugados": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes de tipo C fugados,7.084857,4.807692,28.151105,28.289982,28.42886,58.035714


In [74]:
group = df[df['arrive_time'] < 12].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_morning = {"total de clientes fugados en la mañana": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_morning).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados en la mañana,4.889084,0.0,8.311235,8.407071,8.502907,34.090909


In [75]:
group = df[(df['arrive_time'] >= 12) & (df['arrive_time'] < 14)].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_noon = {"total de clientes fugados al medio día": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_noon).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados al medio día,7.343457,17.307692,45.475413,45.61936,45.763307,70.542636


In [76]:
group = df[(df['arrive_time'] >= 14)].groupby('simulation').agg({
    "leakage": "sum",
    "arrive_time": "count"
})

group['percent'] = group['leakage'] / group['arrive_time'] * 100

leakage_total_tarde = {"total de clientes fugados en la tarde": compute_stats(group, 'percent') }

pd.DataFrame(leakage_total_tarde).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados en la tarde,5.294849,2.645503,18.240418,18.344208,18.447998,43.589744


In [79]:
group = df.groupby('simulation').agg({
    "price": "sum",
})

money = {"total de ingresos": compute_stats(group, 'price', formatter=formatear_moneda) }

pd.DataFrame(money).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de ingresos,"$2,962,309.45","-$7,772,175.77","$7,121,156.50","$7,179,223.73","$7,237,290.96","$17,397,989.76"


In [80]:
group = df[df['type'] == "A"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_A = {"Tiempo promedio en el sistema clientes tipo A": compute_stats(group, 'mean') }

pd.DataFrame(system_time_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo A,0.062403,0.208369,0.378591,0.379814,0.381038,0.660764


In [81]:
group = df[df['type'] == "B"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_B = {"Tiempo promedio en el sistema clientes tipo B": compute_stats(group, 'mean') }

pd.DataFrame(system_time_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo B,0.035012,0.14879,0.260931,0.261617,0.262303,0.466237


In [82]:
group = df[df['type'] == "C"].groupby('simulation').agg({
    "system_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['system_time'] / group['arrive_time'] 

system_time_C = {"Tiempo promedio en el sistema clientes tipo C": compute_stats(group, 'mean') }

pd.DataFrame(system_time_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo C,0.087167,0.167419,0.409105,0.410813,0.412522,0.819532


In [84]:
group = df[~df['waiting_time'].isna() & (df['type'] == "A")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_A = {"Tiempo promedio en la sala de espera clientes tipo A": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_A).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo A,0.061063,0.172691,0.348801,0.349998,0.351195,0.620341


In [85]:
group = df[~df['waiting_time'].isna() & (df['type'] == "B")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_B = {"Tiempo promedio en la sala de espera clientes tipo B": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_B).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo B,0.02359,0.131622,0.215523,0.215985,0.216447,0.351333


In [86]:
group = df[~df['waiting_time'].isna() & (df['type'] == "C")].groupby('simulation').agg({
    "waiting_time": "sum",
    "arrive_time": "count"
})

group['mean'] = group['waiting_time'] / group['arrive_time'] 

waiting_time_C = {"Tiempo promedio en la sala de espera clientes tipo C": compute_stats(group, 'mean') }

pd.DataFrame(waiting_time_C).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo C,0.092605,0.15256,0.378365,0.38018,0.381995,0.817219


# Resultados 

In [93]:
# Estadisticas de la fuga de los clientes

pd.DataFrame({
    **leakage_total,
    **leakage_total_A,
    **leakage_total_B,
    **leakage_total_C, 
    **leakage_total_morning,
    **leakage_total_noon,
    **leakage_total_tarde,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de clientes fugados,3.734115,9.662921,22.724064,22.797261,22.870457,37.887067
total de clientes de tipo A fugados,5.488518,7.623318,25.631532,25.739118,25.846704,48.235294
total de clientes de tipo B fugados,4.012501,0.000000,11.896750,11.975404,12.054057,28.682171
total de clientes de tipo C fugados,7.084857,4.807692,28.151105,28.289982,28.428860,58.035714
total de clientes fugados en la mañana,4.889084,0.000000,8.311235,8.407071,8.502907,34.090909
total de clientes fugados al medio día,7.343457,17.307692,45.475413,45.619360,45.763307,70.542636
total de clientes fugados en la tarde,5.294849,2.645503,18.240418,18.344208,18.447998,43.589744


In [92]:
# Estadisticas de ingresos

pd.DataFrame(money).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
total de ingresos,"$2,962,309.45","-$7,772,175.77","$7,121,156.50","$7,179,223.73","$7,237,290.96","$17,397,989.76"


In [91]:
# Estadisticas de tiempo en el sistema

pd.DataFrame({
    **system_time_A,
    **system_time_B,
    **system_time_C,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en el sistema clientes tipo A,0.062403,0.208369,0.378591,0.379814,0.381038,0.660764
Tiempo promedio en el sistema clientes tipo B,0.035012,0.148790,0.260931,0.261617,0.262303,0.466237
Tiempo promedio en el sistema clientes tipo C,0.087167,0.167419,0.409105,0.410813,0.412522,0.819532


In [90]:
# Estadisticas de tiempo en la lista de espera

pd.DataFrame({
    **waiting_time_A,
    **waiting_time_B,
    **waiting_time_C,
}).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Tiempo promedio en la sala de espera clientes tipo A,0.061063,0.172691,0.348801,0.349998,0.351195,0.620341
Tiempo promedio en la sala de espera clientes tipo B,0.023590,0.131622,0.215523,0.215985,0.216447,0.351333
Tiempo promedio en la sala de espera clientes tipo C,0.092605,0.152560,0.378365,0.380180,0.381995,0.817219


In [89]:
# Estadisticas de los trabajadores

group = df.groupby('simulation').agg({
    'worker_helper': [
        ('0', lambda x: (x == 0).sum()/ x.count() * 100), 
        ('1', lambda x: (x == 1).sum()/ x.count() * 100), 
        ('2', lambda x: (x == 2).sum()/ x.count() * 100), 
        ('3', lambda x: (x == 3).sum()/ x.count() * 100)  
    ],
})


group.columns = ['_'.join(col).strip() for col in group.columns.values]

workers = {
    "Porciento de clientes atendidos por el modulo 1": compute_stats(group, 'worker_helper_0'), 
    "Porciento de clientes atendidos por el modulo 2": compute_stats(group, 'worker_helper_1'), 
    "Porciento de clientes atendidos por el modulo 3": compute_stats(group, 'worker_helper_2'), 
    "Porciento de clientes atendidos por el modulo 4": compute_stats(group, 'worker_helper_3'), 
}

pd.DataFrame(workers).transpose()

,desviación estándar,mínimo,mínimo del intervalo de confianza,media,máximo del intervalo de confianza,máximo
Porciento de clientes atendidos por el modulo 1,2.037945,16.112532,24.047788,24.087736,24.127684,31.521739
Porciento de clientes atendidos por el modulo 2,2.305480,19.113573,28.567019,28.612211,28.657403,39.823009
Porciento de clientes atendidos por el modulo 3,2.131347,12.827988,20.888317,20.930095,20.971874,28.342246
Porciento de clientes atendidos por el modulo 4,2.316167,16.893733,26.324556,26.369958,26.415360,36.729223
